# Question 1: Structural features
**1.1 cif-cn-featurizer**

Our friend Anton Oliynyk has recently released a new python script `cif-cn-featurizer`. 

Here is their description "A Python script designed to process CIF (Crystallographic Information File) files and extract various features from them. These features include interatomic distances, atomic environment information, and coordination numbers. The script can handle binary and ternary compounds."

Let's test it out and see how it works! To do so we will need some cifs. Right now the featurizer only works with binaries made up of certain elements shown in this plot. 

![allowed elements](https://github.com/sp8rks/MaterialsInformatics/blob/main/HW/HW2/cif-cn-featurizer-allowed-elements.png?raw=true)

**<font color='teal'>a)</font>** Download the `cif-cn-featurizer` files and run it on the cif files in the `HW\cn-featurizer\cifs` folder. 

Note: in case you can't get it working, you'll also find a csv folder with all the extracted features for these cifs already complete, but try and get it working so you can use it in the future!

In [1]:
#you can run this in your miniconda command prompt if you prefer

c:\Users\taylo\miniconda3\envs\MyPymatgen\lib\site-packages\pymatgen\ext\matproj_legacy.py:164: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


**1.2 Getting labeled data for the cifs**

**<font color='teal'>b)</font>** Now that you've got feature vectors in a series of .csv files, let's use them to build a model to predict a property. To get a property let's search for a materials project entry using the cif cards! If you've forgotten how, go back to the `legacy_MPRester_tutorial.ipynb` notebook where we did an example. Once you have the material project id, run a query to extract a property like bulk modulus (["elasticity"]["K_VRH"])

In [ ]:
#your code goes here

**1.3 Comparing structural features to compositional features**

**<font color='teal'>c)</font>** Now that you've got structural features and you can get compositional features (use CBFV), let's compare them! Build a Support vector machine regressor model with each feature set and determine which works better. 

In [ ]:
#your code goes here

asdf


In [ ]:
#your code goes here

asdf
